# 3) Analyze and evaluate optimization output - TEST PROTOCOLS

This final notebook uses the `runs.pkl` file created in notebook 2 and it analyzes:

- the distance between different strategies in the feature space - test protocols


In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path
from pprint import pprint
from tqdm import tqdm

from scipy.stats import kruskal, mannwhitneyu, wilcoxon

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
save_fig = True
figure_folder = Path(".") / "figures_hay_ais"
# figure_folder = Path("/Users/damart/Desktop/LFP") / "figures"

if save_fig:
    figure_folder.mkdir(exist_ok=True)

In [ ]:
base_dir = Path("../..")

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
feature_sets = {"soma": "soma",
                "all": "extra",
                "sections": "extra",
                "single": "extra"}
figsize = (10, 7)

## Load GT params and optimization output

In [ ]:
# general
model_name = "hay_ais"
probe_type = "planar" # linear 

cell_models_folder = base_dir / "cell_models"
model_folder = cell_models_folder / model_name
probe_file = model_folder / "fitting" / "efeatures" / "probe_BPO.json"

In [ ]:
# result_folder = Path("../../")
result_folder = base_dir / "results" / "220429"

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name,
                                    release=False)
cell_release = mf.create_ground_truth_model(model_name=model_name,
                                            release=True)

probe = mf.define_electrode(probe_file=probe_file)

param_names = [param.name for param in cell.params.values() if not param.frozen]

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
protocol_for_eap = "firepattern_120"

In [ ]:
pkl_file_name = "runs.pkl"

In [ ]:
data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

opt_results_training = None
results_file = f"opt_results_training_{model_name}.pkl"
if (result_folder / results_file).is_file():
    with open(result_folder / results_file, 'rb') as f:
        opt_results_training = pickle.load(f)
else:
    raise Exception(f"Couldn't fint result file: {results_file}. Run notebook 3a first!")

opt_results_test = None
results_test_file = f"opt_results_test_{model_name}.pkl"
if (result_folder / results_test_file).is_file():
    with open(result_folder / results_test_file, 'rb') as f:
        opt_results_test = pickle.load(f)
        compute_responses = False
else:
    compute_responses = True

In [ ]:
compute_responses

In [ ]:
fig, ax = plt.subplots()
min_evals = 3000
color_strategy = {"all": "C1", "sections": "C2", "single": "C3"}

keep_idxs = []
for idx, row in df_model.iterrows():
    if max(row["nevals"]) > min_evals:
        keep_idxs.append(idx)
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=colors_dict[row["strategy"]],
                ls='-', 
                lw=0.8,
                alpha=0.75)
    else:
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=colors_dict[row["strategy"]],
                ls='--', 
                lw=0.5,
                alpha=0.75)

ax.set_title("Min fitness")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlabel("Neval")
ax.set_ylabel("Min fitness")
ax.set_yscale('log')

# Load protocols and original features

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()
extra_kwargs

In [ ]:
protocols_used_for_opt = ["IV_-20", "IV_-100", "IDrest_150", "IDrest_250", "IDrest_300",
                          "APWaveform_260"]

In [ ]:
protocols_to_exclude = ["IV", "APWaveform", "IDrest"]

In [ ]:
print(model_name)

eva_extra = mf.create_evaluator(
    model_name=model_name,
    strategy="all",
    protocols_with_lfp=protocol_for_eap,
    all_protocols=True,
    exclude_protocols=protocols_to_exclude,
    **extra_kwargs
)

In [ ]:
print(f"All test features --> num features {len(eva_extra.fitness_calculator.objectives)}")

# Compute release responses

In [ ]:
t_start = time.time()
responses_release = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                            param_values=params_release)
t_stop = time.time()
print(f"Simulated responses in {np.round(t_stop - t_start, 2)} s")

In [ ]:
eap_release = mf.utils.calculate_eap(responses=responses_release, 
                                     protocols=eva_extra.fitness_protocols, 
                                     protocol_name=protocol_for_eap, **extra_kwargs)

# compute extracellular features
std_from_mean = 0.05
extra_features = mf.efeatures_extraction.compute_extra_features(
    eap_release, fs=extra_kwargs["fs"],
    upsample=extra_kwargs["upsample"])

In [ ]:
features_release = {}
for i in tqdm(np.arange(len(eva_extra.fitness_calculator.objectives)), 
              desc="computing features"):
    obj = eva_extra.fitness_calculator.objectives[i]
    features_release[obj.features[0].name] = {}
    if len(obj.features) == 1:
        feat = obj.features[0]
        feat_value = feat.calculate_feature(responses_release)
        feat_score = feat.calculate_score(responses_release)
        features_release[feat.name]["value"] = feat_value
    else:
        print(f"More than one feature for objective: {obj.name}")

# add extra features
for efeat_name, feat in extra_features.items():
    for chan, feat_val in enumerate(feat):
        fature_name = f"{protocol_for_eap}.MEA.{efeat_name}_{chan}"
        features_release[fature_name] = {}
        features_release[fature_name]["value"] = feat_val

In [ ]:
fig_gt_intra = mf.plot_responses(responses_release, color="k", return_fig=True, max_rows=3)

In [ ]:
fig, ax_fr = plt.subplots()
ax_fr.plot(responses_release["firepattern_200.soma.v"]["time"], 
           responses_release["firepattern_200.soma.v"]["voltage"],
           color="k")
ax_fr.axis("off")

In [ ]:
# fig.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/model.png", dpi=300, transparent=True)

In [ ]:
if save_fig:
    fig_gt_intra.savefig(figure_folder / "gt_intra.pdf")
    #fig_gt_extra.savefig(figure_folder / "gt_extra.pdf")

# Compute and plot best responses

In [ ]:
max_feature_value = 50
if compute_responses:
    opt_results_test = {}

In [ ]:
strategies = ["soma", "all", "sections", "single"]

In [ ]:
if compute_responses:
    for strategy in strategies:
        opt_results_test[strategy] = {}
        print(f"Simulating best '{strategy}' -- seed: {opt_results_training[strategy]['best_seed']}")
        best_params = opt_results_training[strategy]["best_params"]
        t_start = time.time()
        responses = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                            param_values=best_params)
        eap = mf.utils.calculate_eap(responses=responses, protocols=eva_extra.fitness_protocols, 
                                     protocol_name=protocol_for_eap, **extra_kwargs)
        t_stop = time.time()
        print(f"Simulated responses in {np.round(t_stop - t_start, 2)} s")
        eap_release_norm = eap_release / np.ptp(np.abs(eap_release), 1, keepdims=True)
        eap_norm = eap / np.ptp(np.abs(eap), 1, keepdims=True)
        eap_dist = np.sum(np.abs(eap_release_norm.ravel() - eap_norm.ravel()))
        opt_results_test[strategy]["eap_dist"] = eap_dist
        opt_results_test[strategy]["responses"] = responses
        opt_results_test[strategy]["eap"] = eap    

In [ ]:
feat_objectives = [obj.features[0].name for obj in eva_extra.fitness_calculator.objectives]

In [ ]:
if compute_responses:
    for strategy in strategies:
        responses = opt_results_test[strategy]["responses"]
        eap = opt_results_test[strategy]["eap"]
        extra_features_strategy = mf.efeatures_extraction.compute_extra_features(
                                        eap, fs=extra_kwargs["fs"],
                                        upsample=extra_kwargs["upsample"])
        opt_results_test[strategy]["extra_features"] = extra_features_strategy

        features_best = {}
        feat_release_keys = list(features_release.keys())
        for i in tqdm(np.arange(len(feat_release_keys)), desc=f"computing features {strategy}"):

            feat_name = feat_release_keys[i]
            features_best[feat_name] = {}
            
            release_value = features_release[feat_name]["value"]

            if feat_name in feat_objectives:
                feat = eva_extra.fitness_calculator.objectives[feat_objectives.index(feat_name)].features[0]

                feat_value = feat.calculate_feature(responses)
                if feat_value is None:
                    feat_value = max_feature_value

                feat_score = np.abs(release_value - feat_value) / np.abs(std_from_mean * release_value)

            else:
                # extra
                _, _, efeat_full = feat_name.split(".")
                efeat_split = efeat_full.split("_")
                chan = int(efeat_split[-1])
                efeat = "_".join(efeat_split[:-1])

                feat_value = extra_features_strategy[efeat][chan]

                if release_value != 0:
                    feat_score = abs(feat_value - release_value) / abs(std_from_mean * release_value)
                else:                    
                    feat_score = abs(feat_value - release_value)

            features_best[feat_name] = {"value": feat_value, "score": feat_score}

        opt_results_test[strategy]["features"] = features_best

In [ ]:
protocols_to_plot = ["firepattern_200", "HyperDepol_-160", "HyperDepol_-40", 
                     "sAHP_250", "PosCheops_300"]
titles = protocols_to_plot
figs_intra = {}
for strategy in strategies:
    responses_to_plot = [responses_release, opt_results_test[strategy]["responses"]]
    colors = ["k", colors_dict[strategy]]
    labels = ["GT", strategy.upper()]
    fig = mf.plot_multiple_responses(responses_to_plot, 
                                    colors=colors, return_fig=True, 
                                    protocol_names=protocols_to_plot,
                                    titles=titles,
                                    figsize=(7, 12))
    figs_intra[strategy] = fig

In [ ]:
mf.plot_cell(eva_extra.cell_model, eva_extra.sim, param_values=params_release,
             detailed=False, exclude_sections=["soma"], 
             alpha=0.1, color="gray")

In [ ]:
from matplotlib.patches import Ellipse, Rectangle
alpha = 0.3

In [ ]:
figs_extra = {}
axs_extra = {}
for strategy in strategies:
    responses = opt_results_test[strategy]["responses"]
    responses_to_plot = [responses_release, responses]
    colors = ["k", colors_dict[strategy]]
    labels = ["GT", strategy.upper()]
    ax_extra = mf.plot_multiple_eaps(responses_to_plot, 
                                     eva_extra.fitness_protocols, probe,
                                     protocol_name=protocol_for_eap, 
                                     colors=colors, #labels=labels, 
                                     norm=True)
    
    mf.plot_cell(eva_extra.cell_model, eva_extra.sim, param_values=params_release,
                 detailed=False, exclude_sections=["soma"], 
                 ax=ax_extra, alpha=alpha, color="gray")
    ellipse = Ellipse(xy=(0, 0), width=15, height=15, 
                      edgecolor="grey", color="grey", lw=2, alpha=alpha)
    ax_extra.add_patch(ellipse)
    
    ax_extra.set_xlim(-120, 120)
    ax_extra.set_ylim(-210, 815)
    
    fig = ax_extra.get_figure()
    figs_extra[strategy] = fig
    axs_extra[strategy] = ax_extra

In [ ]:
axs_extra["single"].set_xlim(-120, 120)
axs_extra["single"].set_ylim(-210, 815)

In [ ]:
figs_extra[strategy].savefig(figure_folder / f"extra_{strategy}.png", dpi=300)        
figs_extra[strategy].savefig(figure_folder / f"extra_{strategy}.pdf")   

In [ ]:
if save_fig:
    for strategy in figs_intra:
        figs_intra[strategy].savefig(figure_folder / f"intra_{strategy}.png", dpi=300)
        figs_intra[strategy].savefig(figure_folder / f"intra_{strategy}.pdf")
        figs_extra[strategy].savefig(figure_folder / f"extra_{strategy}.png", dpi=300)        
        figs_extra[strategy].savefig(figure_folder / f"extra_{strategy}.pdf")                

## Compare best-fitted models

In [ ]:
order_full = ["soma", "all", "sections", "single"]
order = []
for strategy in order_full:
    if strategy in opt_results_test:
        order.append(strategy)

### Compare features

In [ ]:
feature_name_array = []
feature_set_array = []
feature_score_array = []
feature_type_array = []
protocol_type_array = []

for strategy in strategies:
    feats = opt_results_test[strategy]["features"]
    for feat_name, feat_dict in feats.items():
        feature_set_array.append(strategy)
        feature_name_array.append(feat_name)
        if "MEA" not in feat_name:
            feature_type_array.append("intra")
        else:
            feature_type_array.append("extra")
        feature_score_array.append(feat_dict["score"])
        protocol_type = feat_name.split(".")[0].split("_")[0]
        protocol_type_array.append(protocol_type)
        
df_feats = pd.DataFrame({"feature_set": feature_set_array, "feat_name": feature_name_array,
                         "feature_type": feature_type_array, "feat_score": feature_score_array, 
                         "protocol_type": protocol_type_array})

In [ ]:

df_feats_intra = df_feats.query("feature_type == 'intra'").dropna()
df_feats_extra = df_feats.query("feature_type == 'extra'").dropna()

fig_feat_intra, ax = plt.subplots(figsize=(7, 10))
sns.boxplot(data=df_feats_intra, x="feature_set", y="feat_score", order=order, #hue="protocol_type", 
            ax=ax, showfliers=False)
n = len(df_feats_intra.query("feature_set == 'soma'"))
# g = sns.swarmplot(data=df_feats, y="feature_set", x="feat_score", ax=ax)
ax.set_ylabel("Feature scores", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title(f"Intracellular features\n(n={n})", fontsize=20)
ax.set_xlabel("Strategy", fontsize=15)
ax.set_ylabel("Score", fontsize=15)
#ax.set_ylim(0, 21)

fig_feat_extra, ax = plt.subplots(figsize=(7, 10))
sns.boxplot(data=df_feats_extra, 
            x="feature_set", y="feat_score", order=order, ax=ax, showfliers=False)
n = len(df_feats_extra.query("feature_set == 'soma'"))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title(f"Extracellular features\n(n={n})", fontsize=20)
ax.set_xlabel("Strategy", fontsize=15)
ax.set_ylabel("Score", fontsize=15)
#ax.set_ylim(0, 21)


In [ ]:
if save_fig:
    fig_feat_intra.savefig(figure_folder / "intra_features.pdf")
    fig_feat_extra.savefig(figure_folder / "extra_features.pdf")    

In [ ]:
!pip install statsmodels
!pip install scikit_posthocs
import scipy.stats as ss
import statsmodels.api as sa
import scikit_posthocs as sp

In [ ]:
sp.posthoc_conover(df_feats_intra, val_col='feat_score', 
                   group_col='feature_set', p_adjust = 'holm')

In [ ]:
sp.posthoc_conover(df_feats_extra, val_col='feat_score', 
                   group_col='feature_set', p_adjust = 'holm')

In [ ]:
intra_soma = df_feats_intra.query("feature_set == 'soma'")["feat_score"]
intra_sections = df_feats_intra.query("feature_set == 'sections'")["feat_score"]
intra_all = df_feats_intra.query("feature_set == 'all'")["feat_score"]
intra_single = df_feats_intra.query("feature_set == 'single'")["feat_score"]

extra_soma = df_feats_extra.query("feature_set == 'soma'")["feat_score"]
extra_sections = df_feats_extra.query("feature_set == 'sections'")["feat_score"]
extra_all = df_feats_extra.query("feature_set == 'all'")["feat_score"]
extra_single = df_feats_extra.query("feature_set == 'single'")["feat_score"]

In [ ]:
print("Intra - Sections VS SOMA:", wilcoxon(intra_sections, intra_soma))
print("Intra - All VS SOMA:", wilcoxon(intra_all, intra_soma))
#print("Intra - Single VS SOMA:", wilcoxon(intra_single, intra_soma))

In [ ]:
print("Extra - Sections VS SOMA:", wilcoxon(extra_sections, extra_soma))
print("Extra - All VS SOMA:", wilcoxon(extra_all, extra_soma))
print("Extra - Single VS SOMA:", wilcoxon(extra_single, extra_soma))

In [ ]:
for protocol_type in np.unique(df_feats_intra.protocol_type):
    fig, ax = plt.subplots(figsize=(10, 7))
    df_protocol = df_feats_intra.query(f"protocol_type == '{protocol_type}'")
    n = len(df_protocol.query("feature_set == 'soma'"))
    sns.boxplot(data=df_protocol, 
                y="feature_set", x="feat_score",# hue="protocol_type", 
                order=order, ax=ax)
    ax.set_title(f"{protocol_type}\n(n={n})", fontsize=15)
    ax.set_ylabel("Feature scores", fontsize=12)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
#     ax.set_title("Intracellular features", fontsize=15)
    ax.set_xlim([0, 30])

In [ ]:
if save_fig:
    fig_feat_intra.savefig(figure_folder / "feat_intra.pdf")
    fig_feat_extra.savefig(figure_folder / "feat_extra.pdf")

In [ ]:
df_test = pd.DataFrame.from_dict(opt_results_test, orient="index")
df_test["strategy"] = df_test.index

In [ ]:
fig_cos, ax = plt.subplots()
sns.barplot(data=df_test, x="strategy", y="eap_dist", order=order, ax=ax)
ax.set_ylabel("Distance", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title("Extracellular difference", fontsize=15)

In [ ]:
for strategy in opt_results_test:
    responses = opt_results_test[strategy]["responses"]
    responses_to_plot = [responses_release, responses]
    colors = ["k", colors_dict[strategy]]
    labels = ["GT", strategy.upper()]
    ax_extra = mf.plot_multiple_eaps(responses_to_plot, 
                                     eva_extra.fitness_protocols, probe,
                                     protocol_name=protocol_for_eap, 
                                     colors=colors, labels=labels, norm=True)

In [ ]:
opt_results_test["gt"] = {}
opt_results_test["gt"]["responses"] = responses_release
opt_results_test["gt"]["eap"] = eap_release

In [ ]:
with open(result_folder / results_test_file, 'wb') as f:
    pickle.dump(opt_results_test, f, protocol=pickle.HIGHEST_PROTOCOL)